In [6]:
from tkinter import *
import tkinter as tk
from tkinter.filedialog import askopenfilename
from PIL import Image, ImageTk

In [7]:
def escolhe_arquivo():
    caminho_do_arquivo = askopenfilename(filetypes=[("Arquivo CSV", "*.csv"), ("Arquivo Excel", "*xlsx")])
    print(caminho_do_arquivo)
    return 

In [8]:
raiz = Tk(className='Primeira janela de teste para a interface')
raiz.geometry('960x540')

''

In [9]:
### criando o frame de seleção de arquivos ###
frame1 = Frame(raiz) #bg="blue"
frame1.pack(side='top',fill=tk.BOTH)

arq = PhotoImage(file='iconArq.png', master=frame1)
arq = arq.subsample(2,2)
fig1 = Label(frame1, image=arq)
fig1.image = arq

botao = Button(frame1, image=arq, command=escolhe_arquivo, bd=0, highlightthickness=0, bg= 'green')
botao.pack()

In [10]:
raiz.mainloop()

In [15]:
import tkinter as tk
# CRIANDO A PARTE LATERAL DIREITA DA INTERFACE - BET
# Função para criar a janela
def criar_janela():
    janela = tk.Tk()
    janela.geometry("600x400")

    # Criando a janela principal
    janela.title("mvp projeto")  

    def on_scroll(*args):
        canvas.yview(*args)

    # Criando o frame principal
    frame_principal = Frame(janela)
    frame_principal.pack(side = 'right',pady=20, fill=tk.BOTH)
    scale = tk.Scrollbar(frame_principal, orient=tk.VERTICAL, command=on_scroll)
    scale.pack()

    # Criando o widget na lateral direita
    widget_direita = Label(frame_principal, text="lateral esquerda", padx=40, pady=10, borderwidth=2, relief="solid")
    widget_direita.pack(fill=tk.Y, expand=True)

    # Adicionando mais widgets ou funcionalidades conforme necessário...
    

    janela.mainloop()  # Iniciando o loop principal da janela

# Chamando a função para criar a janela
criar_janela()